<a href="https://colab.research.google.com/github/sribarrow/audiorecording/blob/master/cnn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Conv2D, MaxPooling2D, Dropout
from keras.layers import Dense, Activation, Flatten
from keras import optimizers, regularizers

Using TensorFlow backend.


Reference: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=3IEVK-KFxi5Z

In [5]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from tensorflow.python.client import device_lib

In [7]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12927811161125809439, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8195325464865065260
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11413248780337535628
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12985650763744672478
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# with open('/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat '/gdrive/My Drive/foo.txt'

Reference: https://github.com/tensorflow/tensorflow/issues/17076

Fix for ERROR - report_tensor_allocations_upon_oom

In [0]:
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 1} ) 
#  sess = tf.Session(config=config) 
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [11]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [12]:
from tensorflow.core.protobuf import config_pb2
sess.run(c, options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))

array([[22., 28.],
       [49., 64.]], dtype=float32)

Reference: https://stackoverflow.com/questions/46066850/understanding-the-resourceexhaustederror-oom-when-allocating-tensor-with-shape

Fix for ERROR - ResourceExhaustedError: OOM when allocating tensor with shape

In [0]:
gpu_options = tf.GPUOptions(allow_growth=True)

In [0]:
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [0]:
train_path = '/gdrive/My Drive/raag_images/train'
valid_path = '/gdrive/My Drive/raag_images/check'
test_path = '/gdrive/My Drive/raag_images/test'

In [16]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path,  target_size=(300,300), classes=['RaagM','RaagS'],  batch_size=10)

Found 576 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [17]:
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(300,300,3)))
keras.layers.ConvLSTM2D(128, (3, 3), strides=(1, 1), padding='same')
model.add(TimeDistributed(Dense(32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
#model.add(Dense(128, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizers.Adam(lr=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [19]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 300, 32)      1056      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 150, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 720000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 720000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1440002   
Total params: 1,441,954
Trainable params: 1,441,954
Non-trainable params: 0
____________________________________________

In [20]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
print(STEP_SIZE_TRAIN)
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size
print(STEP_SIZE_VALID)


18
6


In [21]:
model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=10, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
18/18 [==============================] - 177s 10s/step - loss: 7.9865 - acc: 0.5035 - val_loss: 8.7306 - val_acc: 0.4583
Epoch 2/10
18/18 [==============================] - 7s 382ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 3/10
18/18 [==============================] - 7s 379ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 6.5938 - val_acc: 0.5909
Epoch 4/10
18/18 [==============================] - 7s 369ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.7917 - val_acc: 0.4545
Epoch 5/10
18/18 [==============================] - 7s 370ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.7917 - val_acc: 0.4545
Epoch 6/10
18/18 [==============================] - 7s 369ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
18/18 [==============================] - 7s 379ms/step - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 

In [22]:
model.evaluate_generator(generator=valid_batches, steps=STEP_SIZE_VALID)

[7.387460390726726, 0.5416666666666666]

**Fine tuning the model using pretrained model VGG16**

In [0]:
from keras.layers import Input

In [0]:
from keras.layers.pooling import AveragePooling2D

In [0]:
from keras.applications.vgg16 import VGG16

In [0]:
#vgg16_model = keras.applications.vgg16.VGG16()
#vgg16_model.layers.pop()

In [27]:
vgg16_model = keras.applications.vgg16.VGG16(include_top=False, input_tensor=Input(shape=(300, 300, 3)))

58892288/58889256 [==============================] - 3s 0us/step


In [28]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

Fine tuning Reference: https://www.pyimagesearch.com/2019/06/24/change-input-shape-dimensions-for-fine-tuning-with-keras/

**Print the type of this vgg16 model.**

*Note this is of type model and not sequential*


In [29]:
type(vgg16_model)

keras.engine.training.Model

In [0]:
new_model = vgg16_model.output

In [31]:
new_model = AveragePooling2D(pool_size=(4, 4))(new_model)

In [0]:
new_model = Flatten(name="flatten")(new_model)
new_model = Dense(128, activation="relu")(new_model)
new_model = Dropout(0.5)(new_model)
new_model = Dense(2, activation="softmax")(new_model)

In [0]:
from keras.models import Model
model = Model(inputs=vgg16_model.input, outputs=new_model)

In [34]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0   

**Transform the above model to sequential**

In [0]:
for l in vgg16_model.layers:
  layer_trainable=False

In [36]:
seq_model = Sequential()
type(seq_model)
print(seq_model)

**Add layers from vgg16 to our model**

In [37]:
for l in model.layers:
  print(l)
  seq_model.add(l)

**Remove output layer and change it to fit previous**

In [38]:
seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 75, 75, 256)      

In [0]:
opt = optimizers.Adam(lr=0.001)

seq_model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [40]:
seq_model.fit_generator(train_batches, steps_per_epoch=STEP_SIZE_TRAIN, epochs=5, validation_data=valid_batches, validation_steps=STEP_SIZE_VALID, verbose=2)

Epoch 1/5
 - 49s - loss: 7.9104 - acc: 0.4948 - val_loss: 8.7306 - val_acc: 0.4583
Epoch 2/5
 - 37s - loss: 8.0590 - acc: 0.5000 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 3/5
 - 36s - loss: 8.0590 - acc: 0.5000 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 4/5
 - 36s - loss: 8.0590 - acc: 0.5000 - val_loss: 7.3264 - val_acc: 0.5455
Epoch 5/5
 - 36s - loss: 8.0590 - acc: 0.5000 - val_loss: 8.0590 - val_acc: 0.5000


In [41]:
seq_model.evaluate_generator(generator=valid_batches, steps=STEP_SIZE_VALID)

[8.05904769897461, 0.5]

In [0]:
session.close()